In [23]:
import numpy as np
from dateutil.parser import parse

# 指定打开的文件名
# 不需要的行需要skip掉
# 默认没有分隔符，所以需要指定 delimiter
# 不加载全部的情况下需要指定加载哪些列 usecols
# 希望把每列加载到单独的数组中，需设置 unpack=True，并指定对应的变量名，列举下有 unpack 和没有的区别
# stock_info = np.loadtxt('./20191019_babaStock.csv', delimiter=',', usecols=(1, 2, 3, 4, 5), skiprows=1)
stock_info = np.loadtxt('./20191019_babaStock.csv', delimiter=',', usecols=(1, 2, 3, 4, 5), skiprows=1, unpack=True)

stock_info.shape
stock_info = stock_info[:, ::-1]
stock_info

# 上涨下跌的天数
close_info = stock_info[0]
open_info = stock_info[2]
# 上涨
rise_count = len(close_info[open_info - close_info > 0])
# rise_count = close_info[open_info - close_info > 0].size
# print('rise count:' + str(rise_count))
# 下跌
fail_count = len(close_info[open_info - close_info <= 0])
# print('fail count:' + str(fail_count))
# 上张天数占比
rise_count / close_info.size
close_info

# 日线转换为周线
# 什么是周线
high_info = stock_info[3]
low_info = stock_info[4]

# loadtxt 方法有 converters 参数，可利用自定义的函数把 string 做转换
def convert_date(d):
    return parse(d).weekday()
# 将日期转换为 weekday 

stock_info = np.loadtxt('./20191019_babaStock.csv', delimiter=',', usecols=(0, 1, 3, 4, 5),
                        skiprows=1, dtype='S', converters={0: convert_date})
#print(stock_info)

# 倒序排列
stock_info = stock_info[::-1, :].astype('f8')
# 需要按照每周去分组
# 先找到星期一的数据的索引
week_split = np.where(stock_info[:, 0] == 0)[0]
# 按照周一去分组，split 返回给定索引的分组
# 可以指定任意间隔的索引，所以 split 以一个 list 的形式返回
week_infos_temp = np.split(stock_info, week_split)
# print(type(week_infos_temp))

# 为了简单起见，只使用一周数据有五天的
week_infos_temp
week_info = [ x for x in week_infos_temp if len(x) == 5 ]
# 再把数据转换成 ndarray
w = np.array(week_info)
# print(w.shape)
# print(w[:3])
week_open = w[:, 0, 3]
print("周线的开盘价：", week_open[:3])
week_close = w[:, -1, 1]
print("周线收盘价：", week_close[:3])
week_high = w[:, :, 3].max(axis=1)
print("周线最高价：", week_high[:3])

week_low = w[:, :, 4].min(axis=1)
print("周线最低价", week_low[:3])

w_info = np.array([week_open, week_close, week_high, week_close])
#print(w_info[:3])
# 一周的数据放到一行，可以直接用转置矩阵
print('周线 result: \n', w_info.T)

# 结果保存到文件
np.savetxt('./20191019_babaStock_result.cvs', w_info.T, header='open, close, high, low', delimiter=',', fmt='%.2f')

周线的开盘价： [79.12 79.84 77.  ]
周线收盘价： [79.89 76.94 79.2 ]
周线最高价： [85.89 79.84 79.94]
周线最低价 [77.71 75.66 75.01]
周线 result: 
 [[ 79.12    79.89    85.89    79.89  ]
 [ 79.84    76.94    79.84    76.94  ]
 [ 77.      79.2     79.94    79.2   ]
 [ 79.98    77.16    80.485   77.16  ]
 [ 79.93    78.79    80.2     78.79  ]
 [ 78.33    75.92    78.53    75.92  ]
 [ 76.455   77.      78.7686  77.    ]
 [ 78.7899  76.29    79.46    76.29  ]
 [ 81.9     81.25    82.88    81.25  ]
 [ 82.79    84.49    85.      84.49  ]
 [ 84.79    82.48    84.79    82.48  ]
 [ 84.8     84.59    85.08    84.59  ]
 [ 85.02    98.25    98.35    98.25  ]
 [ 98.86    96.86    98.86    96.86  ]
 [ 96.63    95.06    96.63    95.06  ]
 [101.38   104.64   105.39   104.64  ]
 [105.5    107.71   109.76   107.71  ]
 [107.06   105.79   109.87   105.79  ]
 [106.29   106.     107.55   106.    ]
 [109.     101.85   109.     101.85  ]
 [103.47   103.94   104.99   103.94  ]
 [105.3    101.93   105.3    101.93  ]
 [103.21    97.57   1